# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features['B']
logdis = np.log(boston_features['DIS'])
loglstat = np.log(boston_features['LSTAT'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns = ['target'])
type(X)

pandas.core.frame.DataFrame

### Train-test split

Perform a train-test split with a test set of 20%.

In [3]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [6]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [10]:
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [11]:
from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print(f'Training MSE {train_mse}')
print(f'Test MSE {test_mse}')

Training MSE 21.948229964237374
Test MSE 21.114400087117502


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [19]:
def kfolds(data, k):
    """return a list of k subsets of data"""
    # Force data as pandas DataFrame
    # add 1 to fold size to account for leftovers
    
    split_size = len(data) // k 
    split_remainder = len(data) % k
    
    folds = []
    start = 0
    end = 0
    
    for i in range(k):
        if split_remainder > 0:
            end = start + split_size + 1
            split_remainder -= 1
        else:
            end = start + split_size
        folds.append(data[start:end])
        start = end
    
    return folds

### Apply it to the Boston Housing data

In [32]:
# Make sure to concatenate the data again
bos_data = pd.concat([boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']], y], axis=1)
bos_data

,CHAS,RM,DIS,B,LSTAT,target
0,0.0,6.575,0.542096,1.000000,-1.275260,24.0
1,0.0,6.421,0.623954,1.000000,-0.263711,21.6
2,0.0,7.185,0.623954,0.989737,-1.627858,34.7
3,0.0,6.998,0.707895,0.994276,-2.153192,33.4
4,0.0,7.147,0.707895,1.000000,-1.162114,36.2
...,...,...,...,...,...,...
501,0.0,6.593,0.331081,0.987619,-0.169811,22.4
502,0.0,6.120,0.297277,1.000000,-0.274682,20.6
503,0.0,6.976,0.274575,1.000000,-1.067939,23.9
504,0.0,6.794,0.315551,0.991301,-0.836660,22.0


In [33]:
# Apply kfolds() to bos_data with 5 folds
folds = kfolds(bos_data, 5)
folds

[     CHAS     RM       DIS         B     LSTAT  target
 0     0.0  6.575  0.542096  1.000000 -1.275260    24.0
 1     0.0  6.421  0.623954  1.000000 -0.263711    21.6
 2     0.0  7.185  0.623954  0.989737 -1.627858    34.7
 3     0.0  6.998  0.707895  0.994276 -2.153192    33.4
 4     0.0  7.147  0.707895  1.000000 -1.162114    36.2
 ..    ...    ...       ...       ...       ...     ...
 97    0.0  8.069  0.475885  1.000000 -1.555067    38.7
 98    0.0  7.820  0.475885  0.991502 -1.829759    43.8
 99    0.0  7.416  0.475885  1.000000 -0.912931    33.2
 100   0.0  6.727  0.379096  0.994604 -0.213444    27.5
 101   0.0  6.781  0.390808  0.996672 -0.555806    26.5
 
 [102 rows x 6 columns],      CHAS     RM       DIS         B     LSTAT  target
 102   0.0  6.405  0.369415  0.177720 -0.012136    18.6
 103   0.0  6.137  0.369415  0.993873  0.378596    19.3
 104   0.0  6.167  0.321174  0.989384  0.235000    20.1
 105   0.0  5.851  0.262627  0.992814  0.717270    19.5
 106   0.0  5.836  0.2

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [56]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
#     train = pd.concat(folds[:n-1] + folds[n+1:])
    train = pd.concat([fold for i, fold in enumerate(folds) if i!=n])
    print(len(train))
    test = folds[n]
    # Fit a linear regression model
    linreg = LinearRegression()
    linreg.fit(train[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']], train['target'])
    y_hat_train = linreg.predict(train[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']])
    y_hat_test = linreg.predict(test[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']])
    # Evaluate Train and Test errors
    train_mse = mean_squared_error(train['target'], y_hat_train)
    test_mse = mean_squared_error(test['target'], y_hat_test)
    train_errs.append(train_mse)
    test_errs.append(test_mse)

print(train_errs)
print('')
print(test_errs)
print('')
print(f'Mean train error {sum(train_errs) / len(train_errs)}')
print(f'Mean test error {sum(test_errs) / len(test_errs)}')

404
405
405
405
405
[24.195577370388623, 23.032087348477972, 19.745072857978982, 15.31710138425119, 22.32997280754659]

[13.405144922008196, 17.44401680068205, 37.032711390020715, 58.279543847842305, 26.0979887571484]

Mean train error 20.923962353728673
Mean test error 30.451881143540334


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [44]:
from sklearn.model_selection import cross_val_score

In [47]:
linreg = LinearRegression()

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')

In [48]:
cv_5_results

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [50]:
np.mean(cv_5_results)

-30.451881143540316

In [51]:
from sklearn.metrics import  make_scorer

In [52]:
mse = make_scorer(mean_squared_error)

In [53]:
cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse)
cv_5_results

array([13.40514492, 17.4440168 , 37.03271139, 58.27954385, 26.09798876])

##  Summary 

In [ ]:
# the MSE for the test data matches the MSE returned from cross_val_score
# the model seems to be overfitted given the large difference in the MSE of
# the training and testing data sets

In [57]:
def do_some_mse(folds, k):
    test_errs = []
    train_errs = []

    for n in range(k):
        # Split in train and test for the fold
        train = pd.concat([fold for i, fold in enumerate(folds) if i!=n])
        print(len(train))
        test = folds[n]
        # Fit a linear regression model
        linreg = LinearRegression()
        linreg.fit(train[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']], train['target'])
        y_hat_train = linreg.predict(train[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']])
        y_hat_test = linreg.predict(test[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']])
        # Evaluate Train and Test errors
        train_mse = mean_squared_error(train['target'], y_hat_train)
        test_mse = mean_squared_error(test['target'], y_hat_test)
        train_errs.append(train_mse)
        test_errs.append(test_mse)

    print(train_errs)
    print('')
    print(test_errs)
    print('')
    print(f'Mean train error {sum(train_errs) / len(train_errs)}')
    print(f'Mean test error {sum(test_errs) / len(test_errs)}')

In [64]:
do_some_mse(kfolds(bos_data, 10), 10)

455
455
455
455
455
455
456
456
456
456
[22.51122149418271, 23.202722295231602, 23.289877373894324, 21.007886892783375, 21.543949013789277, 20.79416802207609, 22.63496660317758, 14.364781188711396, 21.808532036098836, 22.3289304131356]

[16.040807304777815, 8.619850025204325, 7.642480778773643, 30.60501978620501, 24.769440093599933, 34.78710530085539, 15.12978545918736, 115.22917884713405, 29.59922657376474, 16.342060450546533]

Mean train error 21.348703533308075
Mean test error 29.87649546200488


Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 